<a href="https://colab.research.google.com/github/Farhad-Davaripour/ReAct_Agent_from_Scratch/blob/main/demo.ipynb" target="_parent">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/>
</a>


# Simple ReAct Agentic Workflow from Scratch

## Setup

In [ ]:
%pip install openai

In [1]:
import os
import json
import re

# Load the environment variables from .env file
# from dotenv import load_dotenv
# _ = load_dotenv()

# Load the environment variables from Google Colab Secrets
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [ ]:
# Import and initialize OpenAI
from openai import OpenAI
client = OpenAI()

## System Prompt

In [2]:
system_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop, you output an Answer.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available tools are:

retrieve_tasks(priority_level=None):
e.g. retrieve_tasks(priority_level="High")
Retrieves a list of tasks, including their priority and effort estimates.
If a priority level is specified, it returns only the tasks with that priority.

retrieve_resources:
e.g. retrieve_resources
Retrieves a list of resources, including their available hours and skills.

Example session:

Question: Allocate resources to high-priority tasks.

Thought: I need to allocate resources to high-priority tasks. First, I'll retrieve the high-priority tasks.
Action: retrieve_tasks(priority_level="High")
PAUSE

Observation: Retrieved tasks: [{'task': 'Design Database Schema', 'priority': 'High', 'effort': 20}, {'task': 'Develop Machine Learning Model', 'priority': 'High', 'effort': 40}]

Thought: Now I need to retrieve the available resources.
Action: retrieve_resources
PAUSE

Observation: Retrieved resources: [{'resource': 'Alice', 'skill': 'Database Design', 'available_hours': 40}, {'resource': 'Bob', 'skill': 'Machine Learning', 'available_hours': 30}, {'resource': 'Charlie', 'skill': 'API Development', 'available_hours': 25}]

Thought: I've identified two high-priority tasks: "Design Database Schema" and "Develop Machine Learning Model". Alice can be assigned to the "Design Database Schema" task for 20 hours, and Bob can be assigned to the "Develop Machine Learning Model" task for 30 hours.

Answer: Resources have been allocated to high-priority tasks. Alice has been assigned to the "Design Database Schema" task for 20 hours, and Bob has been assigned to the "Develop Machine Learning Model" task for 30 hours.

Note: Always finish with an answer.
""".strip()


## Tools

In [3]:
class Agent:
    def __init__(self, system="", LLM_type="gpt-4o-mini"):
        self.system = system
        self.messages = []
        self.LLM_type = LLM_type
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result, self.messages

    def execute(self):
        completion = client.chat.completions.create(
                        model=self.LLM_type, 
                        temperature=0,
                        messages=self.messages)
        return completion.choices[0].message.content

In [4]:
def retrieve_tasks(priority_level=None):
    """Retrieve tasks filtered by priority level."""
    
    tasks = [
        {"task": "Implement Machine Learning Model", "priority": "High", "effort": 40},
        {"task": "Optimize Database Queries", "priority": "Medium", "effort": 25},
        {"task": "Design User Interface", "priority": "High", "effort": 30},
        {"task": "Set Up CI/CD Pipeline", "priority": "Low", "effort": 20},
        {"task": "Perform Security Audit", "priority": "Medium", "effort": 35}
    ]
    
    if priority_level:
        filtered_tasks = [task for task in tasks if task["priority"].lower() == priority_level.lower()]
        return filtered_tasks

    return tasks

In [5]:
def retrieve_resources(unused_input=None):
    """Simulated external resource retrieval"""
    return [
        {"resource": "Elena", "skill": "Machine Learning", "available_hours": 45},
        {"resource": "Raj", "skill": "Database Optimization", "available_hours": 30},
        {"resource": "Sofia", "skill": "UI/UX Design", "available_hours": 35},
        {"resource": "Liam", "skill": "DevOps", "available_hours": 25},
        {"resource": "Yuki", "skill": "Cybersecurity", "available_hours": 40}
    ]

In [6]:
action_re = re.compile(r'Action:\s*(\w+)(?:\((.*?)\))?')

def query(question, max_turns=10, known_actions={}, LLM_type="gpt-4o"):
    i = 0
    bot = Agent(system_prompt, LLM_type)
    next_prompt = question

    while i < max_turns:
        i += 1
        result, messages = bot(next_prompt)
        print(result)
        
        # Match the action using the updated regex
        actions = [
            action_re.match(a.strip()) 
            for a in result.split('\n') 
            if action_re.match(a.strip())
        ]
       
        if actions:
            action, action_input = actions[0].groups()
            
            if action_input:
                action_input = action_input.strip("'\"")
                key_value_match = re.match(r'(\w+)=(.+)', action_input)
                if key_value_match:
                    action_input = key_value_match.group(2).strip("'\"")
            else:
                action_input = None

            print(f" -- running {action} with input: {action_input}")

            observation = known_actions[action](action_input)
            
            print("Observation:", observation)
            next_prompt = f"Observation: {observation}"
        else:
            break 

    return messages, result

## Execution

In [7]:
known_actions = {
    "retrieve_tasks": retrieve_tasks,
    "retrieve_resources": retrieve_resources,
}

In [8]:
question = "Allocate resources to high-priority tasks."

In [9]:
model_type = "gpt-4o"
max_turns = 5

In [10]:
messages, result = query(question, max_turns, known_actions, model_type)

Thought: I need to allocate resources to high-priority tasks. First, I'll retrieve the high-priority tasks.

Action: retrieve_tasks(priority_level="High")
PAUSE
 -- running retrieve_tasks with input: High
Observation: [{'task': 'Implement Machine Learning Model', 'priority': 'High', 'effort': 40}, {'task': 'Design User Interface', 'priority': 'High', 'effort': 30}]
Thought: Now I need to retrieve the available resources to see who can be assigned to these high-priority tasks.

Action: retrieve_resources
PAUSE
 -- running retrieve_resources with input: None
Observation: [{'resource': 'Elena', 'skill': 'Machine Learning', 'available_hours': 45}, {'resource': 'Raj', 'skill': 'Database Optimization', 'available_hours': 30}, {'resource': 'Sofia', 'skill': 'UI/UX Design', 'available_hours': 35}, {'resource': 'Liam', 'skill': 'DevOps', 'available_hours': 25}, {'resource': 'Yuki', 'skill': 'Cybersecurity', 'available_hours': 40}]
Thought: I've identified two high-priority tasks: "Implement Mac

In [11]:
messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop, you output an Answer.\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available tools are:\n\nretrieve_tasks(priority_level=None):\ne.g. retrieve_tasks(priority_level="High")\nRetrieves a list of tasks, including their priority and effort estimates.\nIf a priority level is specified, it returns only the tasks with that priority.\n\nretrieve_resources:\ne.g. retrieve_resources\nRetrieves a list of resources, including their available hours and skills.\n\nExample session:\n\nQuestion: Allocate resources to high-priority tasks.\n\nThought: I need to allocate resources to high-priority tasks. First, I\'ll retrieve the high-priority tasks.\nAction: retrieve_tasks(priority_level="High")\nPAUSE\n\nObservatio

In [12]:
print(result)

Thought: I've identified two high-priority tasks: "Implement Machine Learning Model" and "Design User Interface". Elena, who has a skill in Machine Learning and 45 available hours, can be assigned to the "Implement Machine Learning Model" task, which requires 40 hours. Sofia, who has a skill in UI/UX Design and 35 available hours, can be assigned to the "Design User Interface" task, which requires 30 hours.

Answer: Resources have been allocated to high-priority tasks. Elena has been assigned to the "Implement Machine Learning Model" task for 40 hours, and Sofia has been assigned to the "Design User Interface" task for 30 hours.
